json: Standard Python library to handle JSON formatting.

boto3: AWS SDK for Python. Used to interact with AWS services (like S3)


In [ ]:
import boto3
import json

# Creates an S3 client so the Lambda function can perform S3 operations (e.g., copy files).
s3_client = boto3.client('s3')

# This is the main entry point for any Lambda function.
# event contains details of what triggered the Lambda (in this case, an S3 file upload).
# context provides runtime info (not used here)

def lambda_handler(event, context):
    
    # Extracts the source bucket name and file key (file name) from the event JSON payload.
    # This event structure is sent by S3 when a new file is uploaded.
    source_bucket  = event['Records'][0]['s3']['bucket']['name']
    object_key = event['Records'][0]['s3']['object']['key']

    # Sets the target bucket (where you want to copy the file).
    # Prepares a copy source dictionary needed by the copy_object() call.
    target_bucket = 'copy-of-raw-json-intermediate-bucket'
    copy_source = {'Bucket': source_bucket, 'Key': object_key}

    # Uses an S3 waiter to pause execution until the uploaded file is fully available.
    # This prevents copying a file that's still being uploaded (especially important for large files)
    waiter = s3_client.get_waiter('object_exists')
    waiter.wait(Bucket=source_bucket, Key=object_key)

    # Executes the copy operation from the source to the target S3 bucket.
    s3_client.copy_object(Bucket=target_bucket, Key=object_key, CopySource=copy_source)

    # Returns a success response for debugging and logging.
    # Not essential for the S3 trigger to work, but helpful for tracing in CloudWatch.
    return{
        'statusCode': 200,
        'body': json.dumps('Copy completed successfully')
    }